## 前言建议

通用的贝叶斯建模流程：
1. 当研究问题时，以一个简化模型开始，并以清晰、好理解、实证无误的代码实现它，将注意力集中在好的建模决策而不是优化上（这同样符合奥卡姆剃刀原则）；
2. 一旦简化模型有效，再找到最大的错误来源，这可能需要增加离散近似过程中值的数量，或者增加蒙特卡洛方法中的迭代次数，或者增加模型细节；
3. 如果对你的应用而言性能已经足够了，则没必要继续优化，但是如果要优化，有两个方向可以考虑：
    1. 评估代码寻找性能的可能优化空间；
    2. 发现找到计算捷径的分析方法；
    
一个基本观点是：近似的良好的模型 >> 精确但是糟糕的模型；

# 贝叶斯定理

## 概率

0到1之间的数字（包括0和1）,1表示肯定发生，0表示肯定不发生，其余表示一个确定性，例如抛硬币正面朝上为0.5；

## 条件概率

在某种**前提条件**下的概率的，例如：

    抛一枚硬币正面朝上的可能性，这个是概率，但是如果抛两枚硬币在第一枚是正面朝上的情况下，第二枚依然是正面朝上的可能性，这就是条件概率;
    
    一般用P(A|B)表示在前提条件B下A事件发生的概率；

## 联合概率

指多个事件同时发生的可能性：

    P(A和B)表示A，B事件都发生的概率，有P(A和B) == P(A)*P(B) -- 公式并不总是成立，前提是各事件为独立事件
    
所谓独立事件A，B，也就是说存在以下关系：

    P(A) == P(A|B)，即A发生的概率与是否有条件B无关
    
对于事件之间有关系的情况，比如下雨（假设如果今天下雨，那么明天也很有可能下雨）有以下关系：

    P(明天下雨|今天下雨) > P(明天下雨)，即如果确定今天下雨了，那么明天下雨的概率就略高于某一天下雨的概率
    
因此通常联合概率表示如下：

    P(A和B) = P(A)*P(B|A)，事件A和B一起发生的联合概率等于事件A的概率*事件B在条件A下的条件概率

对于下雨的例子，如果任意一天下雨概率为0.5，那么P(今天下雨，明天下雨)的概率就可能比0.25大一点；

## 曲奇饼问题

问题：有两个碗，碗1中有30个香草曲奇，10个巧克力曲奇，碗2中各有20个，问随便拿一个曲奇，从碗1取到香草的概率是多少？

    分析：一看就是联合概率问题，有以下公式：
    P(香草 和 碗1) = P(碗1)*P(香草|碗1)
                  = 0.5 * 3/4
                  = 3/8
                  
    上述分析是错误的，正确如下：
    其实是一个条件概率问题，求P(碗1|香草)，也就是拿到的是香草的前提下从碗1中获取；
    设：饼干来自碗1用B1代表，香草用V代表：
    P(V)*P(B1|V) = P(B1)*P(V|B1) - 联合概率是乘积可交换的
    P(B1|V) = P(B1)*P(V|B1)/P(V) = (0.5 * (3/4)) / (5/8) = 0.6
    

## 贝叶斯定理 -- 提供了一个从p(A|B)到p(B|A)的策略

贝叶斯定理的一种应用：由**P(A|B)计算出P(B|A)**，例如上述曲奇饼问题，当P(B1|V)的计算不够显而易见，而P(V|B1)的计算很简单时，该方法很有效；

    推导：
    p(A and B) = p(B and A)
    p(A and B) = p(A)*p(B|A)
    由于A和B并没明确指定，因此可以随意互换：
    p(B)*p(A|B) = p(A)*p(B|A)
    p(A|B) = p(A)*p(B|A)/p(B)

怎么理解p(碗1|香草) = p(碗1)\*p(香草|碗1)/p(香草)：

1. 如果增加碗的个数，那么p(碗1)会降低，同时整个的概率也会降低 -- **碗多了**，更难拿到碗1了，符合实际；
2. 如果增加碗1中香草的比例，比如碗1中改为35:5的个数，那么整个概率会升高 -- 碗1中香草多了，更**容易拿到碗1中的香草**，符合实际；
3. 如果增加碗2中香草的比例，比如碗2中同样为30:10的个数，那么整个概率都会降低 -- 碗2中香草多了，更**容易拿到碗2中的香草**，也就是在**拿到香草的前提**下，更**容易从碗2中出现**，因此相对**碗1就更难**了，符合实际；

## 历时诠释

另一种理解贝叶斯定理的思路：它给我们提供的是一种根据数据集D的内容变化更新假设概率H的方法，称之为“历时诠释”；

“历时”意味着某些事情随着时间的推移而发生，在本例中，就是假设的概率随着看到的新数据而改变。

在考虑H和D的情况下，贝叶斯定理表达式可以改写成：**p(H|D)=(p(H)\*p(D|H))/(p(D))**

每一项解释为：
- p(H)称为先验概率，即在得到新数据前某一假设的概率；
- p(H|D)称为后验概率，即在看到新数据后，我们要计算的某个假设的概率，也就是我们最终要求的部分；
- p(D|H)是该假设下得到这一数据的概率，称为似然度；
- p(D)是在任何假设下得到这一数据的概率，称为标准化常量；

曲奇饼例子分析历时诠释的含义：
1. p(H)：先验概率，只看碗，只有两个碗，因此碗1概率为0.5，同样碗2也是0.5；
2. p(D|H)：似然度，假设1前提下的香草概率，碗1里香草概率为0.75，碗2为0.5；
3. p(D)：标准化常量，不看碗，只看香草在总数中的概率，为5/8；
4. p(H|D)：最终要计算的，即在香草味的前提下从碗1中拿到的概率；

## M&M问题

问题：有两袋M&M豆，94袋中30%褐色，20%黄色，20%红色，10%绿色，10%橙色，10%黄褐色，96袋中24%蓝色，20%绿色，16%橙色，14%黄色，13%红色，13%褐色，从两袋中各取一个豆，一个是黄色，一个是绿色，问黄色豆来自94年的袋子的概率是多少？

    分析：黄色豆来自94年的袋子的概率，即求P(94袋|黄色)：
    p(94袋|黄色) = p(94袋)*p(黄色|94袋)/p(黄色)
                = 1/2 * 1/5 / 0.17
                = 58.82%

没错，上述分析还是错的，完全没考虑绿色豆的条件，假设黄豆来自袋1，绿色来自袋2，下面用历时诠释&表格的方式来分析一下：

| 全部假设 | 先验概率p(H)| 似然度p(D\|H) | p(H)\*p(D\|H) | 后验概率p(H\|D) |
| --- | --- | --- | --- | --- |
| 假设A(袋1:94,袋2:96) | 1/2 | (20)\*(20) | 200 | 20/27 |
| 假设B(袋1:96,袋2:94) | 1/2 | (10)\*(14) | 70 | 7/27 |

即：黄豆来自于94的概率为20/27，约等于74%；

注意：后验概率计算中的27来自于全概率，也是p(H)/p(D|H)这一列的和，即200+70；

## Monty Hall问题

蒙梯大厅问题：三扇门，其中一扇后有奖品，你随便挑选一扇，然后会展示剩下两扇中一扇没有奖品的门，然后你再做出选择，是保持原来的选择，还是从新选择另一扇没被打开的门；

In [74]:
import numpy as np
def monty_hall(count = 10000, keep = True):
    win = 0
    for i in range(count):
        prize = np.random.randint(3)
        choose = np.random.randint(3)
        drop = -1
        if choose == prize:
            all = [0,1,2]
            del all[choose]
            drop = all[np.random.randint(2)]
        else:
            all = [0,1,2]
            del all[max(prize,choose)]
            del all[min(prize,choose)]
            drop = all[0]
        if not keep:
            all = [0,1,2]
            del all[max(drop,choose)]
            del all[min(drop,choose)]
            choose = all[0]
        if prize == choose:
            win += 1
    return 1.*win/count
        
print '模拟Monty Hall问题：'
print 'not change, wins:'+str(monty_hall(count=100000))
print 'change, wins:'+str(monty_hall(count=100000,keep=False))

模拟Monty Hall问题：
not change, wins:0.33203
change, wins:0.66929


### 口述简单理解Monty Hall问题为什么change带来这么大的收益

1. 假设没有改变，那么很简单，无论主持人是否剔除了一个错误的选择，你获奖概率总是1/3；
2. 假设改变了选择，也就是改为另外两个中没有被剔除的那个，为什么此时的概率是2/3呢？

        假设我们能选中除最初自己选择外的另外两个，那么概率就将是2/3，其实现在看看当我们改变自己的选择时，虽然表面上看依然是随便选中了一个，但是实际上相当于选中了两个，因为两个中的一个应该被证实没有奖品，所以选不选中也无所谓，说白了还是贝叶斯定理，新的信息（被丢弃的那个没有奖）改变了先验概率；

### 正确分析流程

假设选手选了A门：

元素含义：

* D -- 包含两部分，Monty打开了门B，而且后面没有奖品；
* p(H) -- 先验概率，也就是没有其他条件情况下选中奖品的概率，也就是1/3；
* p(D|H) -- 似然度，代表在H下得到D的概率；
* p(H|D) -- 后验概率，D情况下各个假设的获奖概率；

表格分析：

| 全部假设 | p(H) | p(D &#124; H) | p(H)\*p(D &#124; H) | 归一化总值 | p(H &#124; D) | 
| --- | --- | --- | --- |  --- | --- |
| 假设奖品在A门 | 1/3 | 奖品在A，选手选A，那么Monty可以随机打开B，C中的一个，因此概率为1/2 | 1/6 | 1/2 | (1/6)/(1/2)=1/3 | 
| 假设奖品在B门 | 1/3 | 奖品在B，因为Monty无法打开奖品在的门，因此概率为0 | 0 | 1/2 | 0 | 
| 假设奖品在C门 | 1/3 | 奖品在C，选手选A，那么Monty只能打开B门，因此此时概率为1 | 1/3 | 1/2 | (1/3)/(1/2)=2/3 |

从结果看，如果选手最先选了A门，Monty打开了B门，那么奖品在A门的概率为1/3，而在C门的概率为2/3，也就是说如果选手改变自己的决定为C门，那么获奖概率从33%增加到了66%；

## 讨论

对于很多涉及条件概率的问题，贝叶斯定理提供了一种曲线救国的思路，如果p(A|B)难以计算，或用实验去衡量，那么转变思路，看看p(B|A),p(A),p(B)是否更容易计算；